### weakref 모듈
 -   파이썬에서는  weakref.ref  함수를 사용하여 약한 참조를 만들 수 있ㅅ브니다. 약한 참조란 다른 객체를 참조하되 참조 카운트는 증가시키지 않는 기능입니다.

In [1]:
import weakref

a  = np.array([1,2,3])
b = weakref.ref(a)

b

<weakref at 0x0000017EA6A34BD8; to 'numpy.ndarray' at 0x0000017EA6A34990>

In [2]:
a =None
b

<weakref at 0x0000017EA6A34BD8; dead>

### weakref 추가

In [3]:
import weakref

class Function(object):
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
            
        self.inputs = inputs
        ## 수정 부분 ========================================================================================================
        self.outputs = [weakref.ref(output) for output in outputs]
        ## ==================================================================================================================
        return  outputs if len(outputs) > 1 else outputs[0]
    

In [4]:
# 세대 추가
class Variable:
    
    
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} 은(는) 지원하지 않습니다.')
                
                
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
        


    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if  f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x : x.generation)
                
        add_func(self.creator)        
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                
                if x.grad is None: 
                    x.grad = gx
                    
                else:
                    x.grad = x.grad + gx
                    
                if x.creator is not None:
                    add_func(x.creator)
                    
                    

class Square(Function):
    def forward(self, x):
        y = x**2
        return y

    def backward(self, gy):
        
        x = self.inputs[0].data
        gx = 2 * x*gy
        return gx
    
    

def square(x):
    return Square()(x)    
    
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        return gy, gy
    
    
def as_array(x):
    if np.isscalar(x):
        return np.array((x))
    return x
def add(x0, x1):
    return Add()(x0, x1)
    
                    

## 동작확인

In [5]:
for i in range(10):
    x = Variable(np.random.randn(10000))
    y = square(square(x))
    


- for 문이 두번째 반복될때  x 와 y가 덮어 써집니다. 그러면 사용자는 이전의 계산 그래프를 더 이상 참조하지 않게 되고 참조 카운트가 0 이 되므로 이 시점에 계산 그래프에 사용된 메모리가 바로 삭제됩니다.


# 메모리 절약 모드

In [3]:
# 세대 추가
class Variable:
    
    
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} 은(는) 지원하지 않습니다.')
                
                
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
        


    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
    # 추가 부분 ===============================================
    def backward(self, retain_grad = False):
    # ========================================================
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if  f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x : x.generation)
                
        add_func(self.creator)        
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                
                if x.grad is None: 
                    x.grad = gx
                    
                else:
                    x.grad = x.grad + gx
                    
                if x.creator is not None:
                    add_func(x.creator)
                    
                    
            ### 추가 부분 =====================================
            if not  retain_grad:
                for y in f.outputs:
                    y().grad = None #y 는 약한 참조(weakref)
                    

In [4]:
# 이와 같이 ndarray 인스턴스는 참조 카운트 방식에 따라  메로리에서 삭제 됩니다.

In [5]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

NameError: name 'add' is not defined

- 신경망에는 학습(training) 과 추론(inference)라는 두 가지 단계가 있습니다. 학습 시에는 미분값을 구해야 하지만, 추론 시에는 단순히 순전파만 하기 대문에 중간 계산결과를 곧바로 버리면 메모리 사용량을 줄일 수 있습니다.

In [6]:
class Config:
    enable_backprop = True

In [7]:
import weakref

class Function(object):
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        # 추가 부분 ===================================================================================
        if Config.enable_backprop: # config  데이터는 한 군데에만 존재하는게 좋기때문에 인스턴스화 하지 않고, 클래스 상태로 이용합니다.
        # ===============================================================================================
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)

            self.inputs = inputs
            ## 수정 부분 ========================================================================================================
            self.outputs = [weakref.ref(output) for output in outputs]
            ## ==================================================================================================================
        return  outputs if len(outputs) > 1 else outputs[0]
    

### 모드전환

In [8]:
Config.enable_backprop = True
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))
y.backward()



Config.enable_backprop = False
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))
y.backward()

NameError: name 'square' is not defined

### with 문을 활용한 모드 전환
 - 파이썬에서  with라고 하는, 후처리를 자동으로 수행하고자 할 대 사용할 수 있는 구문이 있습니다.
 

In [9]:
f = open('sample.txt', 'w')
f.write('hello word!')
f.close()

# 보다시피 open()으로 파일을 열고, 무언가를 쓴 다음 , close()로 파일을 닫습니다. 이때 매번 close() 하기란 귀찮기도 하거니와
# 깜빡하고 잊어버릴 때도 있습니다. with 문은 이런 실수를 막아 줍니다.

In [10]:
with open('test.txt', 'w') as f: 
    f.write('heel')

In [11]:
f.write('\n sdf')

ValueError: I/O operation on closed file.

In [14]:
import contextlib

@contextlib.contextmanager
def config_test():
    print('start')
    try:
        yield
        
    finally:
        print('done')
        

with config_test():
    print('process...')

start
process...
done


#
- contextlib.contextmanager 데코레이터를 달면 문맥을 판단하는 함수가 만들어 집니다. 그리고 이 함수 안에서 yield 전에는 전처리 로직을 , yield 다음에는 후처리 로직을  작성합니다.

In [20]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
        
    finally:
        setattr(Config, name, old_value)

In [21]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)
    
    

NameError: name 'square' is not defined